# Optimize with Hyperactive
[Home](https://simonblanke.github.io/hyperactive-documentation/)

## Install


In [ ]:
!pip install hyperactive

## Sample strategy

In [1]:
import talib.abstract as ta

from lettrade import DataFeed, Strategy, crossover, crossunder
from lettrade.exchange.backtest import ForexBackTestAccount, let_backtest


class SmaCross(Strategy):
    ema1_period = 9
    ema2_period = 21

    def indicators(self, df: DataFeed):
        df["ema1"] = ta.EMA(df, timeperiod=self.ema1_period)
        df["ema2"] = ta.EMA(df, timeperiod=self.ema2_period)

        df["signal_ema_crossover"] = crossover(df.ema1, df.ema2)
        df["signal_ema_crossunder"] = crossunder(df.ema1, df.ema2)

    def next(self, df: DataFeed):
        if len(self.orders) > 0 or len(self.trades) > 0:
            return

        if df.signal_ema_crossover.l[-1]:
            price = self.data.close.l[-1]
            self.buy(size=0.1, sl=price - 0.001, tp=price + 0.001)
        elif df.signal_ema_crossunder.l[-1]:
            price = self.data.close.l[-1]
            self.sell(size=0.1, sl=price + 0.001, tp=price - 0.001)


lt = let_backtest(
    strategy=SmaCross,
    datas="example/data/data/EURUSD_5m_0_10000.csv",
    account=ForexBackTestAccount,
)

### Example

In [2]:
from hyperactive import Hyperactive
from hyperactive.optimizers import HillClimbingOptimizer


# define the model in a function
def params_parser(args):
    params = [
        ("ema1_period", int(args["ema1_period"])),
        ("ema2_period", int(args["ema2_period"])),
    ]
    return params


def result_parser(result):
    return result["equity"]


model = lt.optimize_model(
    params_parser=params_parser,
    result_parser=result_parser,
    fork_data=True,
)

# search space determines the ranges of parameters you want the optimizer to search through
search_space = {
    "ema1_period": list(range(5, 25, 1)),
    "ema2_period": list(range(10, 50)),
}

optimizer = HillClimbingOptimizer(epsilon=0.1, distribution="laplace", n_neighbours=4)

# start the optimization run
hyper = Hyperactive()
hyper.add_search(model, search_space, optimizer=optimizer, n_iter=1000)
hyper.run()

[0] _optimize_model (Hill Climbing): 100%|──────────| 1000/1000 [01:41<00:00,  9.87it/s, best_iter=0, best_pos=[9 0], best_score=1183.88] 



Results: '_optimize_model'  
   Best score: 1183.88  
   Best parameter set:
      'ema1_period' : 14.0  
      'ema2_period' : 10.0  
   Best iteration: 0  
 
   Random seed: 304816968  
 
   Evaluation time   : 100.14153289794922 sec    [99.89 %]
   Optimization time : 0.11161208152770996 sec    [0.11 %]
   Iteration time    : 100.25314497947693 sec    [9.97 iter/sec]
 


In [ ]:
df = hyper.search_data(model)
df

## Plot

In [ ]:
import plotly.io as pio

pio.renderers.default = "notebook"
pio.templates.default = "plotly_dark"

### Type 1

In [ ]:
import plotly.express as px

fig = px.density_heatmap(
    df,
    x="ema1_period",
    y="ema2_period",
    z="score",
    nbinsx=30,
    nbinsy=30,
    histfunc="max",
    color_continuous_scale="Viridis",
)
fig.show()

### Type 2

In [ ]:
import plotly.express as px

fig = px.density_contour(
    df,
    x="ema1_period",
    y="ema2_period",
    z="score",
    histfunc="max",
)
fig.update_traces(contours_coloring="fill", contours_showlabels=True)
fig.show()